# Llama 3.2 1B 모델 추론 (EndPoint Deployment)
## 추론방법 2: Endpoint Name에서 찾아와 추론.

In [1]:
import boto3
import json


In [2]:
# endpoint list 조회
!aws sagemaker list-endpoints   --name-contains "llama"   --output json | jq -r '.Endpoints[].EndpointName'

/bin/bash: line 1: jq: command not found
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='utf-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [12]:

# SageMaker Runtime 클라이언트 생성
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name='us-west-2')


# 엔드포인트 이름
endpoint_name = "basic-jumpstart-deploy-llama3-2-1b-01"

## 데이터 준비
추론을 위한 데이터 준비. llama 포맷에 맞추기 위한 템플릿과 모델 파라미터 설정

In [16]:
def test(question):  
    template = """
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>\n{instruction}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
    """
    prompt = template.format(instruction=question)

    generation_config = {
        "max_new_tokens": 256,
        "eos_token_id": 128009,          # Llama 3.2 공식 EOS 토큰[1][3]
        "pad_token_id": 128009,          # 패딩 토큰 통일[4]
        "repetition_penalty": 1.2,       # 반복 문구 억제[3]
        "temperature": 0.6,    
        "do_sample": True,
        "top_p": 0.9,
        # 'details': True,
    }
    
    prompt_payload = {
        'inputs':prompt,
        'parameters': generation_config
    }

    
    # JSON 형식으로 데이터 직렬화
    payload = json.dumps(prompt_payload)
    
    # 엔드포인트 호출
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,  # 엔드포인트 이름
        Body=payload,               # 입력 데이터
        ContentType='application/json'  # 데이터 형식
    )
    
    # 응답 처리
    result = json.loads(response['Body'].read().decode('utf-8'))
    print("Model Response:", result['generated_text'])


## 추론 테스트

In [17]:
test("너의 이름은 무엇이야?")

Model Response:  오마이널


In [19]:
test("너의 이름은 무엇이야?")

Model Response:  에이미


In [18]:
test("천연으로 염색하고 싶은데 지속이 꽤 되나요?")

Model Response:  천연로 염색하는 방법에 대한 정보는 약 1000년 이상 전부터 존재합니다. 천연염색제의 개발과 사용은 다양한 시기에서 발생하여, 현재까지 다양한 종류가 존재합니다.

1.  **산화 수소 (Sodium Metaphenoxide)** : 산화수소를 혼합한 물질을 이용해 염색한다.
2.  **산화chlور인스 (Chloroform Oxalate) :** 산화chlorinated 수소와 함께 엄Absolutely 화학적 성분입니다.
3.  **포유사료 (Borax)**: 포유사료를 불aab라 하여 염색할 수 있습니다. 이 proceso에서는 해부학적으로 피부에 들어간 Boron을 바탕으로 포유사료의 고체 형태로 transformed 한 수소 화합물, Boric acid을 제거하는 process도 필요하므로 보통 Boric Acid를 사용하지 않습니다. 

    Boric Acid은 알루민이나 가공식용재료 등에 사용될 때 부정상적인 효과가 있어, 이 용기를 사용하기보다는 B5(카페아민)
